In [1]:
# İlişkisel veritabanları ilişkisel model üzerine kuruludur
# Varlıkları temsil eden tablolardan oluşur, tablonun her satırı söz konusu varlığın bir örneğidir

In [3]:
# SQLite veritabanını ve veritabanına erişmek için SQLAlchemy kullanacağız
# SQLAlchemy, MySQL PostgreSQL gibi pek çok veritabanları içinde kullanılabilir

In [4]:
# İlk olarak sqlalchemy modülünde bulunan create_engine fonksiyonu ile bir SQL engine başlatıyoruz
# Bu motoru kullanarak veritabanı ile etkileşime geçerek sorgularımız icra edeceğiz
# Fonksiyon argüman olarak veritabanı tipi ve veritabanı ismini şu formatta alır:create_engine('sqlite:///db.sqlite')
# Motora işaret eden referansı kullanarak ref.table_names() çağrısı ile tablo isimlerini alabiliriz

In [10]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///chinook.db')
table_names = engine.table_names()
print(table_names)

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'sqlite_sequence', 'sqlite_stat1', 'tracks']


In [11]:
# Sorguları koşabilmek için motora bir bağlantı oluşturmamız gerekir
con = engine.connect()

In [30]:
# Bağlantıyı kurduktan sonra execute metodu ile sorgumuzu koşabiliriz
rs = con.execute('SELECT * FROM artists')

In [39]:
# Sonucu tutan nesneyi dataframe e çevirebiliriz
# Eğer tüm satırları istemiyorsak, rs.fetchmany(size=5) metodunu kullanabiliriz

In [32]:
import pandas as pd

df = pd.DataFrame(rs.fetchall())
df.iloc[:5,:]

,0,1
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [33]:
# Sütun isimlerinin olmadığı görülüyor, bunu düzeltelim

In [35]:
df.columns = rs.keys()
df.iloc[:5,:]

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [36]:
# İşimiz bittikten sonra veritabanı bağlantısını kapatmamız gerekiyor
# Ya da bağlantıyı bir context manager ile açarak bu yükten kurtulabiliriz

In [65]:
con.close()
with engine.connect() as con:
    rs = con.execute('Pragma table_info(employees)')
    columns = [column[1] for column in list(rs)]
    print(columns)

['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email']


In [66]:
# Pandas kütüphanesini kullanarak sorgularımızı daha kısa yapabiliriz

In [70]:
df = pd.read_sql_query('SELECT * FROM employees WHERE EmployeeId >= 6', engine)
df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
1,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
2,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [71]:
df = pd.read_sql_query('SELECT * FROM employees WHERE EmployeeId >= 6 ORDER BY BirthDate', engine)
df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
1,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
2,6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com


In [72]:
# Başka bir örnek, bu sefer INNER JOIN de var

In [81]:
df = pd.read_sql_query('SELECT Title, Name FROM albums INNER JOIN artists on Albums.ArtistId = Artists.ArtistId', engine)
df.head()

,Title,Name
0,For Those About To Rock We Salute You,AC/DC
1,Balls to the Wall,Accept
2,Restless and Wild,Accept
3,Let There Be Rock,AC/DC
4,Big Ones,Aerosmith
